# Extended Kalman filter for Nomoto model
An Extended Kalman filter with a Nomoto model as the predictor will be developed.
The filter is run on simulated data as well as real model test data.

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
import sympy as sp

import vessel_manoeuvring_models.visualization.book_format as book_format
book_format.set_style()
from vessel_manoeuvring_models.substitute_dynamic_symbols import lambdify
from sympy import Matrix
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify
from sympy.physics.vector.printing import vpprint, vlatex
from vessel_manoeuvring_models.data import mdl
from typing import AnyStr, Callable

## Nomoto model for ship manoeuvring dynamics
The Nomoto model can be written as:

In [ ]:
r,r1d,r2d = sp.symbols('r \dot{r} \ddot{r}')
psi,psi1d = sp.symbols('psi \dot{\psi}')
h,u = sp.symbols('h u')
x, x1d = sp.symbols('x \dot{x}')
A,B,C,D,E, Phi = sp.symbols('A B C D E Phi')
w = sp.symbols('w')

K, delta, T_1, T_2 = sp.symbols('K delta T_1 T_2')

eq_nomoto = sp.Eq(K*delta,
                 r + T_1*r1d + T_2*r2d)
Math(vlatex(eq_nomoto))

where $r$ is yaw rate with its time derivatives and $\delta$ is the rudder angle. $K$, $T_{1}$
 and $T_{1}$ are the coefficients describing the hydrodynamics of the ship.
 
For slow manoeuvres this equation can be further simplified by removing the $\ddot{r}$ term into a first order Nomoto model:

In [ ]:
eq_nomoto_simple = eq_nomoto.subs(r2d,0)
Math(vlatex(eq_nomoto_simple))

### Simulation model

In [ ]:
f_hat = sp.Function('\hat{f}')(x,u,w)
eq_system = sp.Eq(x1d, f_hat)
eq_system

Where the state vector $x$:

In [ ]:
eq_x = sp.Eq(x, sp.UnevaluatedExpr(Matrix([psi,r])))
eq_x

and input vector $u$:

and $w$ is zero mean Gausian process noise

For the nomoto model the time derivatives for the states can be expressed as:

In [ ]:
eq_psi1d = sp.Eq(psi1d,r)
eq_psi1d

In [ ]:
eq_r1d = sp.Eq(r1d,sp.solve(eq_nomoto_simple,r1d)[0])
eq_r1d

In [ ]:
def lambda_f_constructor(K, T_1):
    def lambda_f(x, u):
        delta = u
        f = np.array([[x[1], (K*delta-x[1])/T_1]]).T
        return f

    return lambda_f

In [ ]:
jac = sp.eye(2,2) + Matrix([r,eq_r1d.rhs]).jacobian([psi,r])*h
jac

In [ ]:
Matrix([r,
       eq_r1d.rhs]).jacobian([delta])

In [ ]:
def lambda_jacobian_constructor(h,T_1):
    def lambda_jacobian(x, u):

        jac = np.array(
            [
                [1, h],
                [0, 1-h/T_1],
                
            ]
        )
        return jac

    return lambda_jacobian

## Simulation
Simulation with this model where rudder angle shifting between port and starboard

In [ ]:
T_1_ = 1.8962353076056344
K_ = 0.17950970687951323
h_ = 0.02

lambda_f = lambda_f_constructor(K=K_, T_1=T_1_)
lambda_jacobian = lambda_jacobian_constructor(h=h_, T_1=T_1_)

In [ ]:
def simulate(E, ws, t, us):
    
    simdata = []
    x_=np.deg2rad(np.array([[0,0]]).T)
    
    for u_,w_ in zip(us,ws):
               
        x_=x_ + h_*lambda_f(x=x_.flatten(), u=u_)
    
        simdata.append(x_.flatten())
        
    simdata = np.array(simdata)
    df = pd.DataFrame(simdata, columns=["psi","r"], index=t)
    df['delta'] = us
    
    return df

In [ ]:
N_ = 4000

t_ = np.arange(0,N_*h_,h_)
    
us = np.deg2rad(np.concatenate((-10*np.ones(int(N_/4)),
                                10*np.ones(int(N_/4)),
                                -10*np.ones(int(N_/4)),
                                10*np.ones(int(N_/4)))))

np.random.seed(42)
E = np.array([[0, 1]]).T
process_noise = np.deg2rad(0.01)
ws = process_noise*np.random.normal(size=N_)
df = simulate(E=E, ws=ws, t=t_, us=us)

measurement_noise = np.deg2rad(0.5)
df['epsilon'] = measurement_noise*np.random.normal(size=N_)
df['psi_measure'] = df['psi'] + df['epsilon']
df['psi_deg'] = np.rad2deg(df['psi'])
df['psi_measure_deg'] = np.rad2deg(df['psi_measure'])
df['delta_deg'] = np.rad2deg(df['delta'])


In [ ]:
fig,ax=plt.subplots()
df.plot(y='psi_deg', ax=ax)
df.plot(y='psi_measure_deg', ax=ax, zorder=-1)
df.plot(y='delta_deg', ax=ax, zorder=-1)
df.plot(y='r')
ax.set_title('Simulation with measurement and process noise')
ax.set_xlabel('Time [s]');



## Kalman filter
Implementation of the Kalman filter. The code is inspired of this Matlab implementation: [ExEKF.m](https://github.com/cybergalactic/MSS/blob/master/mssExamples/ExEKF.m).

In [ ]:
def extended_kalman_filter(
    x0: np.ndarray,
    P_prd: np.ndarray,
    lambda_f: Callable,
    lambda_jacobian: Callable,
    h: float,
    us: np.ndarray,
    ys: np.ndarray,
    Qd: float,
    Rd: float,
    E: np.ndarray,
    Cd: np.array,
) -> list:
    """Example extended kalman filter

    Parameters
    ----------
    x0 : np.ndarray
        initial state [x_1, x_2]
    P_prd : np.ndarray
        2x2 array: initial covariance matrix

    lambda_f: Callable
        python method that calculates the next time step

        Example:
        def lambda_f(x,u):

            b = 1
            w = 0

            x : states
            u : inputs
            dx = np.array([[x[1], x[1] * np.abs(x[1]) + b * u + w]]).T

        the current state x and input u are the only inputs to this method.
        Other parameters such as b and w in this example needs to be included as local
        variables in the method.

    lambda_jacobian: Callable

        python method that calculates the jacobian matrix

        Example:
        def lambda_jacobian(x, u):

            h=0.1

            jac = np.array(
                [
                    [1, h, 0],
                    [0, 2 * x[2] * h * np.abs(x[1]) + 1, h * x[1] * np.abs(x[1])],
                    [0, 0, 1],
                ]
            )
            return jac

        the current state x and input u are the only inputs to this method.
        Other parameters such as time step h in this example needs to be included as local
        variables in the method.

    h : float
        time step filter [s]
    us : np.ndarray
        1D array: inputs
    ys : np.ndarray
        1D array: measured yaw
    Qd : float
        process noise
    Rd : float
        measurement noise

    Returns
    -------
    list
        list with time steps as dicts.
    """
    x_prd = x0
    time_steps = []

    no_states = len(x0)
    N = len(us)

    for i in range(N):
        t = i * h

        u = us[i]  # input
        y = ys[i].T  # measurement

        # Compute kalman gain:
        S = Cd @ P_prd @ Cd.T + Rd  # System uncertainty
        K = P_prd @ Cd.T @ inv(S)
        IKC = np.eye(no_states) - K @ Cd

        # State corrector:
        P_hat = IKC @ P_prd @ IKC.T + K * Rd @ K.T
        eps = y - Cd @ x_prd
        x_hat = x_prd + K * eps

        # discrete-time extended KF-model
        f_hat = lambda_f(x=x_hat.flatten(), u=u)

        # Predictor (k+1)
        # Ad = I + h * A and Ed = h * E
        # where A = df/dx is linearized about x = x_hat
        Ad = lambda_jacobian(x=x_hat.flatten(), u=u)

        Ed = h * E

        x_prd = x_hat + h * f_hat
        P_prd = Ad @ P_hat @ Ad.T + Ed @ Qd @ Ed.T

        time_step = {
            "x_hat": x_hat.flatten().tolist(),
            "P_prd": P_prd,
            "Ad": Ad,
            "time": t,
            "K": K.flatten().tolist(),
        }

        time_steps.append(time_step)

    return time_steps

In [ ]:
x0=np.deg2rad(np.array([[0,0]]).T)
P_prd = np.diag(np.deg2rad([1, 0.1]))

Qd = np.deg2rad(np.diag([0, 0.5]))

Rd = np.deg2rad(1)

ys = df['psi_measure'].values

E_ = np.array(
    [[0,0], [0,1]],
)

C_ = np.array([[1, 0]])

Cd_ = C_
Ed_ = h_ * E_

time_steps = extended_kalman_filter(x0=x0, P_prd=P_prd, lambda_f=lambda_f, 
                                    lambda_jacobian=lambda_jacobian,h=h_, us=us, ys=ys, E=E_, Qd=Qd, Rd=Rd, Cd=Cd_)
x_hats = np.array([time_step["x_hat"] for time_step in time_steps]).T
time = np.array([time_step["time"] for time_step in time_steps]).T
Ks = np.array([time_step["K"] for time_step in time_steps]).T


In [ ]:
n=len(P_prd)
fig,axes=plt.subplots(nrows=n)

keys = ['psi','r']
for i,key in enumerate(keys):
    
    ax=axes[i]
    df.plot(y=key, ax=ax, label="True")
    if key=='psi':
        df.plot(y='psi_measure', ax=ax, label="Measured", zorder=-1)
    
    ax.plot(time, x_hats[i, :], "-", label="kalman")
    ax.set_ylabel(key)
    ax.legend()

In [ ]:
fig,ax=plt.subplots()
for i,key in enumerate(keys):
    ax.plot(time,Ks[i,:],label=key)
ax.set_title('Kalman gains')
ax.legend();
ax.set_ylim(0,0.1);


# Real data
Using the developed Kalman filter on some real model test data

## Load test

In [ ]:
id=22773
df, units, meta_data = mdl.load(dir_path = '../../data/raw', id=id)
df.index = df.index.total_seconds()
df.index-=df.index[0]

In [ ]:
from vessel_manoeuvring_models.visualization.plot import track_plot
fig,ax=plt.subplots()
fig.set_size_inches(10,10)
track_plot(df=df, lpp=meta_data.lpp, x_dataset='x0', y_dataset='y0',  psi_dataset='psi', beam=meta_data.beam, ax=ax);

In [ ]:
ys = df['psi'].values
h_m=h_ = df.index[1]-df.index[0]
x0=np.deg2rad(np.array([[0,0]]).T)
us = df['delta'].values

P_prd = np.diag(np.deg2rad([1, 0.1]))
Qd = np.deg2rad(np.diag([0, 5]))
Rd = np.deg2rad(0.1)

time_steps = extended_kalman_filter(x0=x0, P_prd=P_prd, lambda_f=lambda_f, 
                                    lambda_jacobian=lambda_jacobian,h=h_, us=us, ys=ys, E=E_, Qd=Qd, Rd=Rd, Cd=Cd_)
x_hats = np.array([time_step["x_hat"] for time_step in time_steps]).T
time = np.array([time_step["time"] for time_step in time_steps]).T
Ks = np.array([time_step["K"] for time_step in time_steps]).T

In [ ]:
n=len(P_prd)
fig,axes=plt.subplots(nrows=n)

    
ax=axes[0]
df.plot(y='psi', ax=ax, label="Measured", zorder=-1)
df['-delta']=-df['delta']
df.plot(y='-delta', ax=ax, label='$-\delta$', zorder=-10)
ax.plot(time, x_hats[0, :], "-", label="kalman", zorder=10)
ax.set_ylabel('$\Psi$')
ax.legend()

ax=axes[1]
ax.plot(time, x_hats[1, :], "-", label="kalman")
ax.set_ylabel('$r$')

ax.legend();